# 미세먼지 농도 예측

* AirkKorea 미세먼지 데이터 활용  
 - 춘천시 석사동 지역의 한달간 미세먼지농도 데이터를 수집
 - 12시간 미세먼지 측정값을 이용하여 1시간 후 미세먼지농도를 예측   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## 미세먼지 dataset

지역 : 석사동
기간 : 20200616 ~ 20200717  
관측값 : 미세먼지(PM10), 초미세먼지(PM2.5), 오존(O3), 이산화질소(NO2), 일산화탄소(CO), 아황산가스(SO2)

예측 : 12시간 이전 값으로 한시간 이후 예측
윈도우 : 입력 : 12시간 데이터, 출력 :  한시간 후 데이터


In [ ]:
csv_path = '/content/drive/My Drive/DeepLearning/data/pmdata_20200617_0717.csv'
#csv_path = './data/pmdata_20200617_0717.csv'
df = pd.read_csv(csv_path)

In [ ]:
# 상위 데이터 목록 확인
df.head()


In [ ]:
#데이터프레임 정보확인
df.info()

In [ ]:
#데이터 통계값 확인
df.describe()

In [ ]:
#datetime index 설정
df.index = df['DateTime']

print(df.head())
print(df.shape)

In [ ]:
#결측치 확인 및 삭제
na = df.isnull().sum()
print("NA = ", na)
df.dropna(inplace=True)
print(df.shape)


In [ ]:
#날짜순으로 정렬
df = df.sort_index(ascending=True)
print(df)

In [ ]:
#미세먼지 값만 추출
uni_data=df["PM10"]
uni_data.index = df['DateTime']
uni_data.head()

print(uni_data.shape)

#그래프로 확인
uni_data.plot(subplots=True)


데이터의 처음 500 개는 학습 데이터 세트고 나머지는 유효성 검사 데이터 세트로 구성

In [ ]:
#학습 데이터 분리를 위한 위치값
TRAIN_SPLIT = 500

재현성을 보장하기 위해 시드 설정.

In [ ]:
tf.random.set_seed(13)

### (3)Standardization  
평균, 표준편차를 구하여 스케일링 처리  
keras.utils.normalize를 이용하여 [0,1] 범위로 재조정 하는 방법을 사용해도됨

In [ ]:
#uni_data에서 값만 추출
uni_data = uni_data.values

#평균, 표준편차 구하기
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

In [ ]:
#Standardization (표준화)
uni_data = (uni_data-uni_train_mean)/uni_train_std

### (4) univariate_data 추출


* univariate_data 함수  
모델이 학습할 때 사용할 window를 반환  
매개 변수 history_size : 과거 정보 window size, target_size : 예측  레이블.

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

* window 추출  
x : 지난 12 개의 PM10 관측값  
y : 1시간 이후 관측값

In [ ]:
univariate_past_history = 12
univariate_future_target = 1

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

print("train data :", x_train_uni.shape)
print("validation data : ", x_val_uni.shape)


In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

### (5) 예측확인 그래프

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'PM10 samples')

### (5) Baseline
모델 학습전에 기준선 결정, 과거 20개 관측값의 평균점을 모델 예측으로 정하여 출력

In [ ]:
def baseline(history):
  return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

## Recurrent neural network

LSTM 을 사용하여 예측모델 구성

### (6) LSTM 입력 데이터 구조 

In [ ]:
BATCH_SIZE = 50
BUFFER_SIZE = 500

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

### (7) LSTM model 구성

In [ ]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),         #output unit size = 8
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')   #regression

### (8) sample prediction

모든 데이터를 예측하여  모델의 출력 결과  확인

In [ ]:
for x, y in val_univariate.take(1):
    pred = simple_lstm_model.predict(x)
print(pred.shape)

In [ ]:
show_plot([x[0].numpy(), y[0].numpy(), pred[0]], 0, 'pre Predicted sample 0')

### (9)모델 학습

In [ ]:
EVALUATION_INTERVAL = 50
EPOCHS = 100

history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=10)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
plot_train_history(history, 'Training and validation loss')

### (10)학습된 LSTM으로 예측결과 확인

In [ ]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()

In [ ]:
# 모델 평가하기
trainScore = simple_lstm_model.evaluate(x_train_uni, y_train_uni, verbose=0)
print('Train Score: ', trainScore)
valScore = simple_lstm_model.evaluate(x_val_uni, y_val_uni, verbose=0)
print('Validataion Score: ', valScore)

## II. 3개 특징으로 예측

In [ ]:
df.info()

In [ ]:
pd.plotting.scatter_matrix(df, s=60, diagonal='kde')

### (1) 3개의 특징으로 구성된 데이터 생성

In [ ]:
features_considered = ['PM10', 'O3', 'NO2']

In [ ]:
features = df[features_considered]
features.index = df['DateTime']
features.head()

In [ ]:
features.plot(subplots=True)

### (2) 표준화 

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [ ]:
dataset = (dataset-data_mean)/data_std

### (3) Single step model
과거기록으로부터 단일지점 예측

step만큼 과거 관측치 샘플링

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [ ]:
past_history = 12
future_target = 1
STEP = 1

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 0], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 0],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))


In [ ]:
BATCH_SIZE = 50
BUFFER_SIZE = 500

train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

### (4)모델 구성

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
#single_step_model.add(tf.keras.layers.Dropout(0.3))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

### (5) sample prediction.

In [ ]:
for x, y in val_data_single.take(1):
  pred = single_step_model.predict(x)
  print(pred.shape)

In [ ]:
show_plot([x[0][:, 1].numpy(), y[0].numpy(), pred[0]], future_target, 'pre Predicted (sample 0)')

### (6)모델 학습

In [ ]:
EVALUATION_INTERVAL = 50
EPOCHS = 20

single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=20)

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

(7) Predict a single step future
12시간 (12 개 데이터 포인트)으로 샘플링 된 3 가지 특징의 예측

In [ ]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 0].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], future_target,
                   'Single Step Prediction')
  plot.show()

  

### 모델 평가

In [ ]:
# 모델 평가하기
trainScore = single_step_model.evaluate(x_train_single, y_train_single, verbose=0)
print('Train Score: ', trainScore)
valScore = single_step_model.evaluate(x_val_single, y_val_single, verbose=0)
print('Validataion Score: ', valScore)
